In [1]:
########################################## FINES STACKER - KOLOMELA ##########################################

In [12]:
#INSTRUCTIONS:
#1. Copy data set to CSV folder to be processed. 
#2. Run this script to generate an array of JSON objects summaizing the CSV files that are available.
#3. Post this array to mongodb.

#Google Drive Functional Spec
#https://docs.google.com/document/d/1DxqJCOKn40OIKej5CaKQ3CNnHxVCJO3I2LFPDUv4V0I/edit#

#User Manual
#https://docs.google.com/document/d/1qL2m3qvlNFPelwpigaVLsu8i0OQEmeqqw3ghwsF5W4E/edit?usp=sharing

In [45]:
#Troubleshooting notes
#1. Install of the pip modules that dont work.
#2. Left of this cell is the the word "In [x]: this defines the cell reference. If this cell is run , and an Asterix * is shown next to it , this implies this is Loading/busy and the user should wait."
#3. The mongouri and ssl certification process changes often. there is generally good stack overflow data to correct when requried.

In [95]:
#Required tests.
#1. check if duration calc is functional.
#2. establish test data set for fines , lumpy and bwr.
#3. Precheck test on 3 data sets prior to posting.

In [133]:
machine = "fines"
path = 'D:/New folder/FINES/FINES - January 2023/06/e/'

In [134]:
#################################################################
########  READ THE CSV FILES TO BE PROCESSED INTO MONGO  ########
#################################################################

############################################
########### Importing Modules ##############
############################################
import glob # module for reading files from a directory
import os # module for getting only the filename not the whole path
import pandas #module pandas
from datetime import datetime #pip module for handling date and time
 
RawData = pandas.DataFrame() # Initialize an empty dataframe 
names = [os.path.basename(x) for x in glob.glob(path+"*.csv", recursive=True)] #loop through all files in the director.
datelist = [] #create empty array for storing CSV data.
 
filecount = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
count = 0    
    
for file_name in glob.glob(path+'*.csv', recursive=True):
    print(file_name +" "+ str(count) + "/" + str(filecount))
    count = count+1
    nameFile = os.path.basename(file_name)#get the file name of the extract only.
    x = pandas.read_csv(file_name, delimiter=';',low_memory=False)#read in each CSV file

    #Extract the start and end times on each CSV file
    startTime= str(x.iloc[[0]]['Time'].values[0]) # string extract of start and end times for CSV
    EndTime= str(x.iloc[[-5]]['Time'].values[0]) # string extract of start and end times for CSV

    #This is code is string managment to allow for proper time keeping in Javascript conversions.
    if startTime[11:13] =='24':
        print(int(startTime[11:13]))
        startTime= str(0)+str(int(startTime[:1])+1)+startTime[2:10]+"_"+str(startTime[11:13].replace("24","00"))+"-"+startTime[14:]
   
    if EndTime[11:13] =='24':
        print(int(EndTime[11:13])+1)
        EndTime= str(0)+ str(int(EndTime[:1])+1)+EndTime[2:10]+"_"+str(EndTime[11:13].replace("24","00"))+"-"+EndTime[14:]
        
    dateObject = str(startTime[0:10]).replace("-", " ")
    timeObject = str(startTime[11:19]).replace("-", " ")

    total = dateObject+' '+timeObject
    dateObject = datetime.strptime(total, '%d %m %Y %H %M %S')
    startTimeStr = str(dateObject)
    startTime = dateObject
   
    dateObject = str(EndTime[0:10]).replace("-", " ")
    timeObject = str(EndTime[11:19]).replace("-", " ")
    total = dateObject+' '+timeObject
    dateObject = datetime.strptime(total, '%d %m %Y %H %M %S')
    EndTimeStr = str(dateObject)
    EndTime = dateObject
    
    from datetime import datetime, timedelta 
    duration = EndTime - startTime

    if duration.days == 1:
        duration=duration-timedelta(days=1)
    if duration.days == -1:
        duration=duration+timedelta(days=1) 
    if duration.days == 2:
        duration=duration-timedelta(days=2)

    #print("----- ------------------------- -----")
    #print("CSV File Name : ",file_name)
    #print('Start Time : ',startTime)
    #print('End Time : ',EndTime) 
    #print("Duration in Days : " ,duration.days)        
    #print("Duration : " ,duration)
    #print("----- ------------------------- -----")
 
    #JSON OBJECT CONSTRUCTION.
    datelist.append({
        "filename":nameFile,
        "startdate":startTimeStr,
        "enddate":EndTimeStr,
        "duration":str(duration),
    })
print('complete')

D:/New folder/FINES/FINES - January 2023/06/e\12-06-2022_04-32-17-576.csv 0/6
D:/New folder/FINES/FINES - January 2023/06/e\12-06-2022_10-32-17-613.csv 1/6
D:/New folder/FINES/FINES - January 2023/06/e\12-06-2022_16-32-17-635.csv 2/6
D:/New folder/FINES/FINES - January 2023/06/e\12-06-2022_22-32-17-675.csv 3/6
D:/New folder/FINES/FINES - January 2023/06/e\13-06-2022_04-32-17-696.csv 4/6
D:/New folder/FINES/FINES - January 2023/06/e\13-06-2022_10-32-17-717.csv 5/6
complete


In [135]:
#This is a file stream function , and as such requries a "promise" for inline code, wont work in the cell as is below. 
from pathlib import Path
from dotenv import load_dotenv # add this line
import os
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

THIS_MACHINE_ENV = os.getenv('THIS_MACHINE_ENV')

if(THIS_MACHINE_ENV != machine):
    print("TEST FAIL : .ENV is incorrect or out of date.")
 

# A missing or out-of-date .env file can cause errors in a software project.
# The .env file stores important configuration information and environment variables needed for the software to function correctly.
# If this information is missing or incorrect, the software may not be able to connect to databases or third-party services, or may not be able to access other critical resources.
# For example, in a Node.js application, if a database connection string is stored in the .env file and the file is missing or contains an outdated connection string, the application will not be able to connect to the database, causing errors or failure to function correctly.
# Similarly, if an API key or other authentication credentials are stored in the .env file and are missing or out-of-date, the software may not be able to authenticate with third-party services or may be denied access to critical resources.
# In summary, the .env file is a critical component of many software projects that store sensitive data and configurations. It should be kept up-to-date and include all necessary environment variables to ensure the software functions correctly.

In [136]:
##############################################
########  CONNECT TO REMOTE MONGODB  ########
############################################
import pymongo #import the pymongo pip
import datetime #pip module for handling date and time
from pymongo import MongoClient #mongodb management
import os
import certifi #pip module for SSL certificates
user = os.getenv('MYSQL_USER')
userPC = os.environ['COMPUTERNAME'] # WORK ONLY ON WINDOWS
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
COLLECTIONCSVFILES = os.getenv('COLLECTIONCSVFILES')
conn_text = 'mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority'.format(user,password,host)
client = MongoClient(conn_text, tlsCAFile=certifi.where())
db = client.entangelment
collection = db[COLLECTIONCSVFILES]
post = {
        "author": userPC,
        "date": datelist,
        "timestamp": datetime.datetime.utcnow(),
        "comment": path
       }
post_id = collection.insert_one(post)
print("Pushed to server : " + path)

Pushed to server : D:/New folder/FINES/FINES - January 2023/06/e/


In [100]:
# https://cloud.mongodb.com/
# The script to follow will upload "datelist" into mongodbcloud.
import webbrowser
webbrowser.open('https://cloud.mongodb.com/')
# The Cluster is under the Isithelo Project , Cluster 
# Entangelment spelt wrong :( 
COLLECTIONCSVFILES = os.getenv('COLLECTIONCSVFILES'); 
print('This CSV datelist file will be pushed to the Mongodb cloud under the collection : ', COLLECTIONCSVFILES , " for the ", THIS_MACHINE_ENV , " machine.")

This CSV datelist file will be pushed to the Mongodb cloud under the collection :  entangelment  for the  fines  machine.


In [23]:
print('This CSV datelist file has been pushed to the Mongodb cloud under the collection : ', COLLECTIONCSVFILES , " for the ", THIS_MACHINE_ENV , " machine.")

This CSV datelist file has been pushed to the Mongodb cloud under the collection :  entangelment  for the  fines  machine.


In [12]:
import webbrowser
webbrowser.open('http://localhost:8888/notebooks/'+machine+'/3 - Process the CSV Files.ipynb')

True